In [1]:
from __future__ import division, print_function

## Get the saved Keras 1.2 model

In [2]:
!rm mnist_cnn_allconv_tensorflow.h5
!wget https://raw.githubusercontent.com/AvantiShri/model_storage/cca175f988023e62b7a2939f151034d38a83c4d3/deeplift/mnist/mnist_cnn_allconv_tensorflow.h5 

rm: mnist_cnn_allconv_tensorflow.h5: No such file or directory
--2018-06-05 22:07:45--  https://raw.githubusercontent.com/AvantiShri/model_storage/cca175f988023e62b7a2939f151034d38a83c4d3/deeplift/mnist/mnist_cnn_allconv_tensorflow.h5
Resolving raw.githubusercontent.com... 151.101.188.133
Connecting to raw.githubusercontent.com|151.101.188.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 979776 (957K) [application/octet-stream]
Saving to: 'mnist_cnn_allconv_tensorflow.h5'

100%[======================================>] 979,776     --.-K/s   in 0.08s   

2018-06-05 22:07:45 (11.1 MB/s) - 'mnist_cnn_allconv_tensorflow.h5' saved [979776/979776]



## Inspect the json and weights of the saved file

In [3]:
import h5py
import json
f = h5py.File("mnist_cnn_allconv_tensorflow.h5")
print("keras version", f.attrs['keras_version'])
print("keras json", json.dumps(json.loads(f.attrs['model_config']),
                               indent=4),"\n")
print("layer weight names:", [layer_name+"/"+x
                              for layer_name in f['model_weights'].keys()
                              for x in f['model_weights'][layer_name].attrs['weight_names']],"\n")

keras version 1.2.0
keras json {
    "class_name": "Sequential", 
    "config": [
        {
            "class_name": "Convolution2D", 
            "config": {
                "W_constraint": null, 
                "b_constraint": null, 
                "name": "convolution2d_1", 
                "activity_regularizer": null, 
                "trainable": true, 
                "dim_ordering": "tf", 
                "nb_col": 4, 
                "subsample": [
                    2, 
                    2
                ], 
                "init": "glorot_uniform", 
                "bias": true, 
                "nb_filter": 32, 
                "activation": "linear", 
                "input_dtype": "float32", 
                "b_regularizer": null, 
                "W_regularizer": null, 
                "nb_row": 4, 
                "batch_input_shape": [
                    null, 
                    28, 
                    28, 
                    1
                ], 
         

/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import keras
print ("keras version",keras.__version__)
import numpy as np

#create a keras 2 model with the same architecture
#set the weights for each layer using the hdf5
#weights file
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(4,4),
                              strides=(2,2),
                              input_shape=(28,28,1)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(4,4),
                              strides=(2,2)))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=10))
model.add(keras.layers.Activation("softmax"))
model.build()

#load the weights into each layer

#first convolution
model.layers[0].set_weights(
    [np.array(f['model_weights']['convolution2d_1/convolution2d_1_W_2:0']),
     np.array(f['model_weights']['convolution2d_1/convolution2d_1_b_2:0'])])
#second convolution
model.layers[2].set_weights(
    [np.array(f['model_weights']['convolution2d_2/convolution2d_2_W_2:0']),
     np.array(f['model_weights']['convolution2d_2/convolution2d_2_b_2:0'])])
#first dense layer
model.layers[6].set_weights(
    [np.array(f['model_weights']['dense_1/dense_1_W_2:0']),
     np.array(f['model_weights']['dense_1/dense_1_b_2:0'])])
#second dense layer
model.layers[9].set_weights(
    [np.array(f['model_weights']['dense_2/dense_2_W_2:0']),
     np.array(f['model_weights']['dense_2/dense_2_b_2:0'])])

Couldn't import dot_parser, loading of dot files will not be possible.
keras version 2.1.6


Using TensorFlow backend.


## Verify the accuracy obtained on MNIST

In [5]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_test = X_test[:,:,:,None]

predictions = model.predict(X_test)
print("accuracy:",float(np.sum(np.argmax(predictions, axis=1)==y_test))/len(y_test))


accuracy: 0.9921


## Save the converted model

In [6]:
model.save("keras2_mnist_cnn_allconv.h5")